
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer                       |

In [ ]:
%number_of_workers 2

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
## Read dataset from S3 e

In [ ]:
dfy_customer= glueContext.create_dynamic_frame_from_options(connection_type= 's3',
                                                               connection_options={"paths": ['s3://adriano-datalake-us-east-1/raw/customer/customer_order/']},
                                                               format='csv', format_options = {"withHeader": True, "optimizePerformance": True})

## Convert string to integer column

In [ ]:
dfy_customer_remapped = ApplyMapping.apply(dfy_customer, mappings =[("order_id", "string", "order_id", "int"),
                                        ("customer_id", "string", "customer_id", "string"),
                                        ("total_amount", "string", "total_amount", "double"),
                                        ("order_date", "string", "order_date", "string")],
                                           transformation_ctx="order_df_transformation")

In [ ]:
dfy_customer_remapped.printSchema()
dfy_customer_remapped.show()

## Filter By Number Range in a Column

In [ ]:
dyf_filtered = Filter.apply(
    frame = dfy_customer_remapped, 
    f = lambda x: x["total_amount"] > 500 )

In [ ]:
dyf_filtered.show()

## Filter Column From Values In a List

In [ ]:
dyf_filtered_2 = Filter.apply(frame = dfy_customer_remapped, 
    f = lambda x: x["order_date"] in ["2022-08-27", "2022-08-28"])
dyf_filtered_2.show()

## Filter Using Multiple Columns

In [ ]:
dyf_filtered_3 = Filter.apply(
    frame = dfy_customer_remapped, 
    f = lambda x: (x["total_amount"] > 500 or x['total_amount'] < 100) and x["order_date"] ==  "2022-08-27")
dyf_filtered_3.show()